In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import (confusion_matrix,roc_curve, auc, accuracy_score, precision_score, recall_score)

In [2]:
n = 250 #number of clusters
n_tags = 1992 #nummber of tags
n_samples = 2210 #number of samples
tagcluster_path1 = "2000tags_cluster250_1.csv"
tagcluster_path2 = "2000tags_cluster250_2.csv"
tagcluster_gmm_path = "2000tags_gmm250.csv"
tagcluster_kmeans_path = "2000tags_kmeans250.csv"
article_path = "sample_seiri_top8.csv"

In [3]:
tagdf = pd.read_csv(tagcluster_gmm_path)
tagdf.drop(range(8),inplace=True)
tagdf.reset_index(inplace=True)
tagdf
# tagdf = tagdf[8:]
# tagdf.sort_values(by="クラスタ番号")

,index,タグ,投稿数,word2vec,fasttext,クラスタ番号
0,8,#音楽,178361,[-0.15375285 -0.83799052 -0.30382207 -0.439428...,[ 2.10155368 -1.81353164 -1.61187506 -10.13...,137
1,9,#毎日更新,171740,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[ 2.34817863e-02 1.76569656e-01 2.36443922e-...,59
2,10,#ビジネス,158846,[-0.12670934 -1.26493979 0.83051515 0.436070...,[ 1.31948829e+00 -8.06355298e-01 -1.26953673e+...,59
3,11,#詩,152937,[-1.93041456 0.34622887 0.30652708 0.039218...,[ 3.9131794 -3.51951098 -3.69753623 -18.06...,62
4,12,#note,138042,[-0.60963827 -0.18487823 0.31421787 -0.029202...,[ 1.03681982 -0.9748475 -0.9645254 -4.32...,62
...,...,...,...,...,...,...
1987,1995,#投資法,2352,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...,[-0.03522037 0.53674728 -0.02258254 -0.295930...,93
1988,1996,#願い,2352,[ 2.04104042 0.7073487 -2.14612412 -3.232718...,[ 2.33439016 -1.5932157 -1.98472297 -9.98...,115
1989,1997,#おめでとう,2351,[-0.05903616 -0.59916937 0.95982933 0.113469...,[ 5.57941794e-01 -5.15634060e-01 -7.65184522e-...,89
1990,1998,#ナイター,2350,[ 2.14836764e+00 -7.23218024e-01 1.31389010e+...,[ 8.38302732e-01 -7.27929294e-01 -8.08724284e-...,82


In [4]:
articledf = pd.read_csv(article_path)
articledf

,title,content,tags,doc_vec,labels
0,№211 みずにながす、がおー。,今日は午前中だけ仕事して、午後医者に行き、夕方はホームレスのお友達からお寿司をおごってもらっ...,['#毎日更新'],"[-0.6413622315797844, -0.02762089093218069, -0...",[9]
1,見送り,父も、母も、もう他界してしばらくが経つ。私も、もう、いい歳で。父も母も、生きていればそれなり...,"['#母', '#父', '#ほぼ毎日note', '#やさしさにふれて']","[-0.21704280540967982, 0.06023082480144997, -0...","[565, 1068, 1146, 1382]"
2,Eテレ『趣味どきっ！』が面白い,最近、たまたま観たのですが『趣味どきっ！』の“スゴ楽” 家事への道シリーズが良かったんです...,"['#習慣', '#テレビ', '#掃除']","[-0.6624432352820425, -0.054146641178606014, -...","[160, 478, 779]"
3,新しい服はしばらく買わないと決めてみて,新しい服を買わないようにしようと決めて3年ほどたった。3年前ぐらいに、断捨離とか、家の整理と...,['#断捨離'],"[-0.6314828735735987, -0.09158760018281703, -0...",[421]
4,蓮の花言葉,父方の祖父母の家には、伯父と伯母と従兄弟が一緒に暮らしていて、台所には冷蔵庫と茶箪笥が2台ず...,"['#介護', '#母', '#認知症', '#父', '#引っ越し']","[-0.522198764722776, -0.01959150434377851, -0....","[199, 565, 890, 1068, 1302]"
...,...,...,...,...,...
2205,レッツダンス❗️,今回、ナゴコンの皆さんと、沢山の方々に助けて頂いて、出来上がって来ました。”緑文化小劇場＆ナ...,"['#ピアノ', '#映像']","[-0.8237114698033441, -0.12204698043113404, -0...","[169, 1051]"
2206,Creepy Nutsの武道館ライブを、配信で見たよ♪,12日夜、Creepy Nuts（クリーピーナッツ）という、ヒップホップのグループの武道...,"['#音楽', '#ライブ配信']","[-0.48791739515120713, -0.38404660005594643, -...","[8, 708]"
2207,珍しいチューナーを見つけた,ども、キャンピングミュージシャンの山木将平です。 最近ギターのチューナーを無くしてしまいどー...,"['#ギター', '#ミュージシャン']","[-0.9868878505362549, -0.032979069012369897, -...","[148, 1399]"
2208,パピコクレさんに捧ぐアリア（歌：Paprikachordさん）,「パピコクレさんに捧ぐアリア」をPaprikachordさんに歌っていただきました。「パピコ...,"['#音楽', '#DTM', '#オリジナル曲']","[-0.9321060173370889, -0.046223862741706555, -...","[8, 103, 488]"


In [5]:
tags = tagdf["タグ"]

In [6]:
articledf["labels"] = [list(np.where(tags.str.match("|".join(s+'$' for s in eval(elem))))[0]) for elem in articledf["tags"]]
articledf

,title,content,tags,doc_vec,labels
0,№211 みずにながす、がおー。,今日は午前中だけ仕事して、午後医者に行き、夕方はホームレスのお友達からお寿司をおごってもらっ...,['#毎日更新'],"[-0.6413622315797844, -0.02762089093218069, -0...",[1]
1,見送り,父も、母も、もう他界してしばらくが経つ。私も、もう、いい歳で。父も母も、生きていればそれなり...,"['#母', '#父', '#ほぼ毎日note', '#やさしさにふれて']","[-0.21704280540967982, 0.06023082480144997, -0...","[557, 1060, 1138, 1374]"
2,Eテレ『趣味どきっ！』が面白い,最近、たまたま観たのですが『趣味どきっ！』の“スゴ楽” 家事への道シリーズが良かったんです...,"['#習慣', '#テレビ', '#掃除']","[-0.6624432352820425, -0.054146641178606014, -...","[152, 470, 771]"
3,新しい服はしばらく買わないと決めてみて,新しい服を買わないようにしようと決めて3年ほどたった。3年前ぐらいに、断捨離とか、家の整理と...,['#断捨離'],"[-0.6314828735735987, -0.09158760018281703, -0...",[413]
4,蓮の花言葉,父方の祖父母の家には、伯父と伯母と従兄弟が一緒に暮らしていて、台所には冷蔵庫と茶箪笥が2台ず...,"['#介護', '#母', '#認知症', '#父', '#引っ越し']","[-0.522198764722776, -0.01959150434377851, -0....","[191, 557, 882, 1060, 1294]"
...,...,...,...,...,...
2205,レッツダンス❗️,今回、ナゴコンの皆さんと、沢山の方々に助けて頂いて、出来上がって来ました。”緑文化小劇場＆ナ...,"['#ピアノ', '#映像']","[-0.8237114698033441, -0.12204698043113404, -0...","[161, 1043]"
2206,Creepy Nutsの武道館ライブを、配信で見たよ♪,12日夜、Creepy Nuts（クリーピーナッツ）という、ヒップホップのグループの武道...,"['#音楽', '#ライブ配信']","[-0.48791739515120713, -0.38404660005594643, -...","[0, 700]"
2207,珍しいチューナーを見つけた,ども、キャンピングミュージシャンの山木将平です。 最近ギターのチューナーを無くしてしまいどー...,"['#ギター', '#ミュージシャン']","[-0.9868878505362549, -0.032979069012369897, -...","[140, 1391]"
2208,パピコクレさんに捧ぐアリア（歌：Paprikachordさん）,「パピコクレさんに捧ぐアリア」をPaprikachordさんに歌っていただきました。「パピコ...,"['#音楽', '#DTM', '#オリジナル曲']","[-0.9321060173370889, -0.046223862741706555, -...","[0, 95, 480]"


In [7]:
#adding clusters column to articledf
articledf["clusters"] = [list(tagdf.loc[s]["クラスタ番号"]) for s in articledf["labels"]]
articledf

,title,content,tags,doc_vec,labels,clusters
0,№211 みずにながす、がおー。,今日は午前中だけ仕事して、午後医者に行き、夕方はホームレスのお友達からお寿司をおごってもらっ...,['#毎日更新'],"[-0.6413622315797844, -0.02762089093218069, -0...",[1],[59]
1,見送り,父も、母も、もう他界してしばらくが経つ。私も、もう、いい歳で。父も母も、生きていればそれなり...,"['#母', '#父', '#ほぼ毎日note', '#やさしさにふれて']","[-0.21704280540967982, 0.06023082480144997, -0...","[557, 1060, 1138, 1374]","[117, 4, 26, 59]"
2,Eテレ『趣味どきっ！』が面白い,最近、たまたま観たのですが『趣味どきっ！』の“スゴ楽” 家事への道シリーズが良かったんです...,"['#習慣', '#テレビ', '#掃除']","[-0.6624432352820425, -0.054146641178606014, -...","[152, 470, 771]","[201, 209, 231]"
3,新しい服はしばらく買わないと決めてみて,新しい服を買わないようにしようと決めて3年ほどたった。3年前ぐらいに、断捨離とか、家の整理と...,['#断捨離'],"[-0.6314828735735987, -0.09158760018281703, -0...",[413],[7]
4,蓮の花言葉,父方の祖父母の家には、伯父と伯母と従兄弟が一緒に暮らしていて、台所には冷蔵庫と茶箪笥が2台ず...,"['#介護', '#母', '#認知症', '#父', '#引っ越し']","[-0.522198764722776, -0.01959150434377851, -0....","[191, 557, 882, 1060, 1294]","[12, 117, 30, 4, 85]"
...,...,...,...,...,...,...
2205,レッツダンス❗️,今回、ナゴコンの皆さんと、沢山の方々に助けて頂いて、出来上がって来ました。”緑文化小劇場＆ナ...,"['#ピアノ', '#映像']","[-0.8237114698033441, -0.12204698043113404, -0...","[161, 1043]","[15, 194]"
2206,Creepy Nutsの武道館ライブを、配信で見たよ♪,12日夜、Creepy Nuts（クリーピーナッツ）という、ヒップホップのグループの武道...,"['#音楽', '#ライブ配信']","[-0.48791739515120713, -0.38404660005594643, -...","[0, 700]","[137, 147]"
2207,珍しいチューナーを見つけた,ども、キャンピングミュージシャンの山木将平です。 最近ギターのチューナーを無くしてしまいどー...,"['#ギター', '#ミュージシャン']","[-0.9868878505362549, -0.032979069012369897, -...","[140, 1391]","[15, 124]"
2208,パピコクレさんに捧ぐアリア（歌：Paprikachordさん）,「パピコクレさんに捧ぐアリア」をPaprikachordさんに歌っていただきました。「パピコ...,"['#音楽', '#DTM', '#オリジナル曲']","[-0.9321060173370889, -0.046223862741706555, -...","[0, 95, 480]","[137, 29, 26]"


In [8]:
X = np.array([np.array(eval(elem)) for elem in articledf["doc_vec"]])
Y = np.array([elem for elem in articledf["labels"]])
Y = MultiLabelBinarizer(classes=range(n_tags)).fit_transform(Y)
# Y = np.array([eval(elem) for elem in articledf["clusters"]])

<ipython-input-8-361b8b867fb9>:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  Y = np.array([elem for elem in articledf["labels"]])


In [9]:
print(X.shape,Y.shape)
print(X)
print(Y)

(2210, 200) (2210, 1992)
[[-0.64136223 -0.02762089 -0.31077059 ... -0.29829604 -0.53742901
   0.18948011]
 [-0.21704281  0.06023082 -0.55525767 ... -0.31878226 -0.48156569
  -0.25063461]
 [-0.66244324 -0.05414664 -0.38273322 ... -0.5921418  -0.41440249
   0.13317313]
 ...
 [-0.98688785 -0.03297907 -0.57405516 ... -0.70934571 -0.44303308
  -0.01264584]
 [-0.93210602 -0.04622386 -0.6457347  ... -0.4526942  -0.18084227
   0.18225742]
 [-0.56223271 -0.12416846 -0.67268819 ... -0.5615323  -0.30514376
   0.01940162]]
[[0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [10]:
indices = np.arange(n_samples)
x_train, x_test, y_train, y_test, idx_train, idx_test = train_test_split(X,Y,indices)

In [11]:
idx_test.shape

(553,)

In [12]:
model = OneVsRestClassifier(GaussianNB())
model.fit(x_train,y_train)

/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 18 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 25 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 26 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 40 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 49 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-

/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 265 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 266 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 267 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 270 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 272 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/

/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 417 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 418 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 420 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 423 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 424 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/

/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 690 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 691 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 698 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 702 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 704 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/

/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 848 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 850 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 851 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 852 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 855 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/

...

/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1186 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1187 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1189 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1190 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1191 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/pytho

/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1379 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1380 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1381 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1382 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1384 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/pytho

/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1613 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1614 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1615 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1616 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1618 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/pytho

/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1840 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1842 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1843 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1844 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/python3.9/site-packages/sklearn/multiclass.py:80: UserWarning: Label not 1846 is present in all training examples.
  warnings.warn("Label %s is present in all training examples." %
/usr/local/lib/pytho

OneVsRestClassifier(estimator=GaussianNB())

In [13]:
y_pred = model.predict(x_test)
y_pred_proba = model.predict_proba(x_test)

In [14]:
def predict(model,x,N=10):
    tmp = model.predict_proba(x)
    pred = np.array([elem.argsort()[-N:][::-1] for elem in tmp])
    return pred
def single_predict(model,x,N=10):
    tmp = model.predict_proba(np.array([x]))
    pred_idx = tmp[0].argsort()[-N:][::-1]
    pred_prob = tmp[0][pred_idx]
    return pred_idx,pred_prob

In [15]:
preddf = articledf.iloc[idx_test] 
preddf

,title,content,tags,doc_vec,labels,clusters
1670,ハエが止まるようなストレートでも,プロ野球の今シーズンが佳境を迎えています。昨今、プロ野球の地上波放送がめっきり減るくらい以前...,"['#プロ野球', '#挑戦している君へ']","[-0.5543698229256177, -0.00930252285287415, -0...","[335, 348]","[48, 147]"
1649,ファン歴１年目のロッテファンが今年のレギュラーシーズンを振り返ってみる,おつかれさまです、こーすけです！！以前、ロッテファンになった経緯について書きました。\n今回...,"['#スポーツ', '#プロ野球']","[-0.05422304426736383, -0.028005311792802023, ...","[96, 335]","[190, 48]"
2207,珍しいチューナーを見つけた,ども、キャンピングミュージシャンの山木将平です。 最近ギターのチューナーを無くしてしまいどー...,"['#ギター', '#ミュージシャン']","[-0.9868878505362549, -0.032979069012369897, -...","[140, 1391]","[15, 124]"
1765,【iPad】タブレットを使ってみた#25,10月末に遂にiPad第8世代が手元にやって来ました。早速色々使ってみた中で、「これは良い」...,"['#note', '#大学生', '#勉強', '#アウトプット', '#オンライン授業'...","[-1.142681080268568, -0.2775294950004411, -1.0...","[4, 17, 64, 173, 697, 891]","[62, 11, 13, 49, 82, 9]"
1416,私を深い沼の中へ手を引っ張ってくれた人、とは,こんにちは、そあれです。やっほい。やっとこさ2つ目の記事です。遅。すまん。ということで(？)...,['#嵐'],"[-0.4313778129472518, 0.03924106645580989, -0....",[1972],[38]
...,...,...,...,...,...,...
1082,最高の企業は「目標」を連鎖させる!? 仕事に関する9つの嘘 その③,こんにちは。Kid.iAです。「世界最重要のビジネス書」として各メディアでも絶賛されている「...,"['#毎日更新', '#ビジネス', '#note', '#仕事', '#毎日投稿', '#...","[-0.6269017405201901, -0.00847021329197997, -0...","[1, 2, 4, 7, 9, 36, 59, 522, 1058]","[59, 59, 62, 134, 89, 183, 59, 101, 93]"
2181,休職2日目 1人飲み,仕事をお休みいただいています。約3週間。今、自宅近くの昼飲みできる居酒屋で1人飲みしています...,"['#旅行', '#海外旅行', '#営業', '#鬱', '#韓国', '#お酒', '#...","[-0.5260444511485365, 0.09284658407766337, -0....","[48, 105, 221, 297, 419, 461, 768, 1035, 1289,...","[128, 89, 172, 104, 45, 111, 93, 134, 93, 9]"
710,勉強嫌いを克服した(い),こんにちは。私は学生時代勉強が大嫌いでした。参考書を開いても全く頭に入ってきませんでしたし、...,['#新社会人'],"[-0.49635982920175814, 0.0057568318408095474, ...",[736],[82]
1452,2020/11/09,最近調子が悪いとグータラってわけじゃ無いのに丸一日起き上がれないなんて日がある。まあまさに昨...,"['#映画', '#写真日記', '#FUJIFILM']","[-0.5518387115847033, -0.009242099041166978, -...","[15, 226, 1369]","[87, 32, 134]"


In [16]:
# from tabulate import tabulate
# predlist = []
# taglist = []
# for idx in range(len(idx_test)):
#   x_test1 = x_test[idx]
#   y_test1 = list(np.where(y_test[idx])[0])
#   pred,prob = single_predict(model,x_test1,N=6)
#   test = y_test1
#   pred = list(pred)
#   taglist.append(list(tagdf.loc[pred,'タグ']))
#   predlist.append(pred)
#   # print("Predicted tags: ",pred)
#   # print("Original tags: ",test)
#   # pred_clusters = np.array([np.array(tagdf["クラスタ番号"][elem]) for elem in pred])
#   # test_clusters = np.array([np.array(tagdf["クラスタ番号"][elem]) for elem in test])

#   # print("Predicted clusters: ",pred_clusters)
#   # print("Original clusters: ",test_clusters)

In [17]:
# preddf['predicted tags'] = taglist
# preddf['predicted labels'] = predlist
# preddf
# # preddf.to_csv("predict_result.csv")

In [18]:
#predict and evaluate one article

#prediction
from tabulate import tabulate
idx = 50
x_test1 = x_test[idx]
y_test1 = list(np.where(y_test[idx])[0])
pred,prob = single_predict(model,x_test1,N=5)
test = y_test1
pred = list(pred)

print("Predicted tags: ",pred)
print("Original tags: ",test)

pred_clusters = np.array([np.array(tagdf["クラスタ番号"][elem]) for elem in pred])
test_clusters = np.array([np.array(tagdf["クラスタ番号"][elem]) for elem in test])

print("Predicted clusters: ",pred_clusters)
print("Original clusters: ",test_clusters)
# print("Predicted clusters: ", list(set(pred_clusters)))
# print("Original clusters: ", list(set(test_clusters)))

#evaluation
lp = len(pred)
lt = len(test)
count_precision = 0
count_recall = 0
count_accuracy = 0

for elem in pred_clusters: #適合率
  if elem in test_clusters:
    count_precision += 1

for elem in test_clusters: #再現率
  if elem in pred_clusters:
    count_recall += 1

if len(test_clusters) == 1: #１個しかついていない文章は合っていれば正解とみなす
  if elem in pred_clusters:
    count_accuracy = 1  
elif count_precision >= 1: #正解率
  count_accuracy = 1

  
precision = count_precision/lp
recall = count_recall/lt
sum = precision + recall
if sum != 0:
  Fscore = 2*precision*recall/(precision + recall)
else:
  Fscore = 0

print("Precision 適合率：", precision)
print("Recall 再現率：", recall)
print("F Score: ", Fscore)
print("Accuracy 正解率: ", count_accuracy)

Predicted tags:  [93, 29, 28, 31, 1]
Original tags:  [200, 577, 593]
Predicted clusters:  [137  26 209   9  59]
Original clusters:  [89  9  9]
Precision 適合率： 0.2
Recall 再現率： 0.6666666666666666
F Score:  0.30769230769230765
Accuracy 正解率:  1


In [19]:
#calculate average F score of test data

from tabulate import tabulate
# F = np.empty([0])
# for n in range(2,11):
n = 5
F = 0
sum_precision = 0
sum_recall = 0
sum_accuracy = 0
for idx in range(len(idx_test)):
  x_test1 = x_test[idx]
  y_test1 = list(np.where(y_test[idx])[0])
  pred,prob = single_predict(model,x_test1,N=n)
  test = y_test1
  pred = list(pred)

  pred_clusters = np.array([np.array(tagdf["クラスタ番号"][elem]) for elem in pred])
  test_clusters = np.array([np.array(tagdf["クラスタ番号"][elem]) for elem in test])

  #evaluation
  lp = len(pred)
  lt = len(test)
  count_precision = 0
  count_recall = 0

  # print("Confsion Matrix:", confusion_matrix(pred_clusters, test_clusters))

  for elem in pred_clusters: #適合率
    # if tagdf["クラスタ番号"][elem] in test_clusters:
    if elem in test_clusters:
      count_precision += 1

  for elem in test_clusters: #再現率
    # if tagdf["クラスタ番号"][elem] in pred_clusters:
    if elem in pred_clusters:
      count_recall += 1

  if len(test_clusters) == 1:
    if elem in pred_clusters:
      sum_accuracy += 1
  elif count_precision >= 2:
    sum_accuracy += 1
    
  precision = count_precision/lp
  recall = count_recall/lt
  sum = precision + recall
  if sum != 0:
    sum_precision += precision
    sum_recall += recall
    F += 2*precision*recall/sum
    # np.append(F, 2*precision*recall/(precision + recall))

  # print("F:", F)
test_length = len(idx_test)
print("Number of Tags: ", n)
print("Average Precision:", sum_precision/test_length)
print("Average Recall:", sum_recall/test_length)
print("Average F Score:", F/test_length)
print("Average Accuracy: ", sum_accuracy/test_length)

Number of Tags:  5
Average Precision: 0.28462929475587695
Average Recall: 0.4469880770360435
Average F Score: 0.31248205316133726
Average Accuracy:  0.72875226039783


In [20]:
articledf["content"][idx_test[idx]]

'世界中のため息を吸い込む長い鼻でも光合成ができる訳じゃない空気を灰色に濁らせたまま少しずつ体が膨張していくのだ今ではパノラマの象は地球よりも大きい体重は測れない、体重計が壊れるから生まれたときは林檎より小さかったのにね星たちが滑り台のように流れていく笑ったり、泣いたり、怒ったり、しながらその一粒は親友で名前は「スニータ」他の星を飲みこんで強くなる性質の星スニータは象の周りをくるくる回り雑学クイズやなぞなぞで遊んでくれる「愛ってなんのこと？」とか「戦争はどうしてなくならないの？」とか哲学的な問いかけには象は答えない考える必要なんてないからね体は肥大してゆくが心は小さく孤独になってゆく寂しさや悲しさは命には付き物だ誰の目にも映るのに、誰も象のことを知らないブラックホールような瞳で太陽と地球の追いかけっこを眺めながら或いは、半分でも見えていれば十分かもしれないもう半分はきっと手遅れなのだスニータがレモンを浮かべた紅茶を飲みたいと呟いた★☆★☆★☆★☆★☆★☆★☆★☆★☆★☆ 読んで頂きありがとうございます最新の詩はこちらのブログから読めます→ http://blog.livedoor.jp/meikasahara/'

In [21]:
articledf["tags"][idx_test[idx]]

"['#詩', '#ポエム']"